In [1]:
import ccxt
import sys 
import os
sys.path.append(os.path.abspath("/Users/vachananand/Documents/Trade Excel/trading_markets"))
import trading_api
import pandas as pd
import time
import numpy as np
from datetime import datetime, timedelta


# Set Parameters per pair

time-frame : {"1m": 1, "5m": 5,"15m": 15, "1h": 60, "1d": 1440}

In [2]:
pair = dict()
pair["name"] = "BATUSDT-XLMUSDT"
pair["symbol_1"] = "BAT/USDT"
pair["symbol_2"] = "XLM/USDT"
pair["window"] = 30
pair["median-win"]=0.03
pair["median-loss"] = -0.0225
pair["cut-loss"] = 0.15
pair["MA"] = "simple"
pair["time-frame"] = "5m"
pair["leverage"] = 1
pair["leg_size"] = 10 # each leg size in USDT
pair["trade_open"] = False # true if 'Open' or 'Filled'


In [3]:
def establish_connection():
    try:
        exchange_id = 'binance'
        exchange_class = getattr(ccxt, exchange_id)
        exchange = exchange_class({
            'apiKey': trading_api.binance["key"],
            'secret': trading_api.binance["secret"],
            'timeout': 30000,
            'enableRateLimit': True

        }) 
        exchange.options["defaultType"] = "future"
        print("Connection Successful !")
        return exchange,True

    except():
        print("Connection NOT Successful")
        return "Connection Declined",False

    
    

In [4]:
def get_historic_close(exchange,pair):
    try:
        data_close_dict = dict()
        for each in range(2):
            data_close = list()
            pair_symbol = pair["symbol_"+str(each+1)]
            pair_timeframe = pair["time-frame"]
            pair_limit = pair["window"] + 1
            data_ohlcv = exchange.fetchOHLCV(symbol=pair_symbol,timeframe=pair_timeframe,limit=pair_limit)
            for each_ohlcv in data_ohlcv:
                data_close.append(each_ohlcv[4])
            data_close_dict[pair["symbol_"+str(each+1)]] = data_close
        close_data = pd.DataFrame(data_close_dict)
        return close_data
    except ccxt.RequestTimeout:
        print("TRY AGAIN, REQUEST TIME OUT")

    

In [5]:
def add_closing_value(pair,exchange,close_data):
    try:
        close_1 = exchange.fetch_ticker(symbol=pair['symbol_1'])["close"]
        close_2 = exchange.fetch_ticker(symbol=pair['symbol_2'])["close"]
        col_1 = pair['symbol_1']
        col_2 = pair['symbol_2']
        temp = close_data[1:]
        close_ser_1 = temp[col_1].append(pd.Series(close_1)).reset_index(drop="index")
        close_ser_2 = temp[col_2].append(pd.Series(close_2)).reset_index(drop="index")
        frame = { col_1: close_ser_1, col_2: close_ser_2 } 
        close_data = pd.DataFrame(frame) 
        return close_data
    except ccxt.RequestTimeout:
        print("TRY AGAIN, REQUEST TIME OUT")

    

In [6]:
def add_current_price(pair,exchange,close_data):
    try :
        close_1 = exchange.fetch_ticker(symbol=pair['symbol_1'])["close"]
        close_2 = exchange.fetch_ticker(symbol=pair['symbol_2'])["close"]
        col_1 = pair['symbol_1']
        col_2 = pair['symbol_2']
        temp = close_data[:-1]
        close_ser_1 = temp[col_1].append(pd.Series(close_1)).reset_index(drop="index")
        close_ser_2 = temp[col_2].append(pd.Series(close_2)).reset_index(drop="index")
        frame = { col_1: close_ser_1, col_2: close_ser_2 } 
        close_data = pd.DataFrame(frame) 
        return close_data
    except ccxt.RequestTimeout:
        print("TRY AGAIN, REQUEST TIME OUT")


    

    

In [7]:
def calc_moving_zscore(spread_ratio,window_size):
    
    #1 day moving average of the price spread
    spreadratio_mavg_1 = spread_ratio.rolling(1).mean()

    # Moving average of the price spread
    spreadratio_mavg_TS = spread_ratio.rolling(window_size).mean()

    # Take a rolling standard deviation
    spreadratio_stdv_TS = spread_ratio.rolling(window_size).std()
    # Compute the z score for each day
    zscore_mvng_ratio = (spreadratio_mavg_1 - spreadratio_mavg_TS)/spreadratio_stdv_TS
        
    return zscore_mvng_ratio,spreadratio_mavg_1,spreadratio_mavg_TS



In [8]:
def get_initial_data(exchange,pair):
    while(1):
        current_utc_time = datetime.utcnow()
        if (current_utc_time.minute % int(pair['time-frame'][:-1]) == 0):
            close_data = get_historic_close(exchange,pair)
            break
    return close_data,current_utc_time
        
    

In [9]:
def get_balance(exchange):
    try:
        balance_wallet = exchange.fapiPrivate_get_balance()
        for each_asset in balance_wallet:
            if each_asset["asset"] == "USDT":
                balance = each_asset["balance"]
                break
                
        return float(balance)

    except:
        print("Retrieving Balance Unsuccessful")
    
    

    

In [10]:
def update_close_data(exchange,inventory,pair):
    try:
        close_data = inventory["close_data"]
        print("UPDATING AFTER "+pair['time-frame'])
        close_data = add_closing_value(pair,exchange,close_data)
        inventory["current_candle_timestamp"] = datetime.utcnow()
        inventory["close_data"] = close_data
        return inventory
    except ccxt.RequestTimeout:
        print("TRYING AGAIN, REQUEST TIME OUT")

    

In [11]:
def get_position(curr_zscore):
    position = "W A T C H"
    if curr_zscore > 2:
        position = "sell"
    if curr_zscore < -2:
        position = "buy"
    return position

In [12]:
def trigger_limit_buy(exchange,pair,log,inventory,close_data):
    
    log["trigger_price_S1"] = close_data.loc[len(close_data)-1,pair['symbol_1']]
    log["trigger_price_S2"] = close_data.loc[len(close_data)-1,pair['symbol_2']]
    if log["enter_trade"]:
        log["position"] = "SHORT"
        log["trigger_amount_S1"] = pair['leg_size']/log["trigger_price_S1"]
        log["trigger_amount_S2"] = log["trigger_amount_S1"] * log["trigger_price_S1"] / log["trigger_price_S2"]
        log["trigger_time"] = datetime.utcnow()
        log["enter_trade"] = False
    elif log["exit_trade"]:
        log["trigger_time"] = datetime.utcnow()
    buy_order = exchange.create_order(symbol=pair['symbol_2'],type="limit", side= "buy", amount=log["trigger_amount_S2"], price=log["trigger_price_S2"])
    inventory["open_positions"][pair['name'].split("-")[1]] = buy_order
    return log,inventory

    
    
    

In [13]:
def trigger_limit_sell(exchange,pair,log,inventory,close_data):
    if True:
        print("SELL SELL SELL")
        log["trigger_price_S1"] = close_data.loc[len(close_data)-1,pair['symbol_1']]
        log["trigger_price_S2"] = close_data.loc[len(close_data)-1,pair['symbol_2']]
        if log["enter_trade"]:
            log["position"] = "LONG"
            log["trigger_amount_S1"] = pair['leg_size']/ log["trigger_price_S1"]
            log["trigger_amount_S2"] = log["trigger_amount_S1"] * log["trigger_price_S1"] / log["trigger_price_S2"]
            log["trigger_time"] = datetime.utcnow()
            log["enter_trade"] = False
        elif log["exit_trade"]:
            log["trigger_time"] = datetime.utcnow()
        sell_order = exchange.create_order(symbol=pair['symbol_2'],type="limit", side= "sell", amount=log["trigger_amount_S2"], price=log["trigger_price_S2"])
        inventory["open_positions"][pair['name'].split("-")[1]] = sell_order
        return log,inventory


In [14]:
def check_entry_status(exchange,pair,inventory,log):
    if True:
        status = "NEW"
        if len(inventory["open_positions"]) > 0 :
            symbol_2 = pair["name"].split("-")[1]
            trade_id = inventory["open_positions"][symbol_2]["id"]
            order = exchange.fapiPrivate_get_order(params={'symbol': symbol_2 ,'orderId': trade_id})
            status = order["status"]
            if status == "FILLED":
                inventory["filled_positions"][pair['name'].split("-")[1]] = order
                inventory["filled_price_S2"] = order["avgPrice"]
                del inventory["open_positions"][pair['name'].split("-")[1]] 
                inventory["num_trade"] += 1
                if not log["exit_trade"]:
                    log["filled_S2"] = True
                else:
                    log["filled_S2"] = False
        return status,inventory,log
    
    
                
    
    
    
    

In [15]:
def trigger_market_sell(exchange,pair,log,inventory):
    if True:
        sell_order = exchange.create_order(symbol=pair['symbol_1'],type="market", side= "sell", amount=log["trigger_amount_S1"])
        inventory["filled_positions"][pair['name'].split("-")[0]] = sell_order
        if log["filled_S2"] and (not log["exit_trade"]):
            log["filled_S1"] = True
            log["entry_time"] = datetime.utcnow()
        else:
            log["filled_S1"] = False
            log["exit_time"] = datetime.utcnow()
            log["enter_trade"] = True
            log["exit_trade"] = False
            log["position"] = ""
            pair["trade_open"] = False
            balance = get_balance(exchange)
        return log,inventory
    

In [16]:
def trigger_market_buy(exchange,pair,log,inventory):
    if True:
        buy_order = exchange.create_order(symbol=pair['symbol_1'],type="market", side= "buy", amount=log["trigger_amount_S1"])
        inventory["filled_positions"][pair['name'].split("-")[0]] = buy_order
        if log["filled_S2"] and (not log["exit_trade"]):
            log["filled_S1"] = True
            log["entry_time"] = datetime.utcnow()
        else:
            log["filled_S1"] = False
            log["exit_time"] = datetime.utcnow()
            log["enter_trade"] = True
            log["exit_trade"] = False
            log["position"] = ""
            pair["trade_open"] = False
            balance = get_balance(exchange)
        return log,inventory
    

In [17]:
def reset_log(log):
    log["position"] = "" # long or short
    log["enter_trade"] = True
    log["exit_trade"] = False
    log["entry_time"] = ""
    log["exit_time"] = ""
    log["connection"] = True
    log["trigger_price_S1"] = -1
    log["trigger_price_S2"] = -1
    log["trigger_amount_S1"] = -1
    log["trigger_amount_S2"] = -1
    log["filled_price_S1"] = -1
    log["filled_price_S2"] = -1
    log["trigger_time"] = ""
    return log

In [18]:
def trigger_market_order(exchange,pair,log,inventory):
    if not log["exit_trade"]:
        if log["position"]=="SHORT":
            log,inventory = trigger_market_sell(exchange,pair,log,inventory)
        else:
            log,inventory = trigger_market_buy(exchange,pair,log,inventory)
        log["exit_trade"] = True
    elif log["exit_trade"]:
        if log["position"]=="SHORT":
            log,inventory = trigger_market_buy(exchange,pair,log,inventory)
        else:
            log,inventory = trigger_market_sell(exchange,pair,log,inventory)
        log = reset_log(log)
    inventory["num_trade"] += 1

    return log,inventory
        
        


In [19]:
def check_revert(exchange,pair,inventory,log,curr_zscore,close_data):
    
    if log["position"] == "LONG" and curr_zscore > 0 and (len(inventory["open_positions"]) == 0):
        log,inventory = trigger_limit_buy(exchange,pair,log,inventory,close_data)
    elif log["position"] == "SHORT" and curr_zscore < 0 and (len(inventory["open_positions"]) == 0):
        log,inventory = trigger_limit_sell(exchange,pair,log,inventory,close_data)
    return pair,inventory,log
        
        

In [20]:
def trade_crypto(exchange,pair,inventory,log):
    stop = False
    
    
    if pair['trade_open']:
        status,inventory,log = check_entry_status(exchange,pair,inventory,log)
        if status == "FILLED" and ( log["filled_S1"] !=log["filled_S2"] ):
            log,inventory = trigger_market_order(exchange,pair,log,inventory)
            
    
    
    
    update_time = inventory["current_candle_timestamp"] + timedelta(minutes = int(pair['time-frame'][:-1]))
    if datetime.utcnow() > update_time:
        inventory = update_close_data(exchange,inventory,pair)
        
    close_data = inventory["close_data"]
    close_data = add_current_price(pair,exchange,close_data)

    
    ratio=close_data[pair['symbol_1']]/close_data[pair['symbol_2']]
    
    zscore_mvng_ratio,_,_ = calc_moving_zscore(ratio,pair["window"])
    
    curr_ratio = ratio[len(ratio)-1]
    curr_zscore = zscore_mvng_ratio[len(zscore_mvng_ratio)-1]
    
    if log["exit_trade"]:
        pair,inventory,log = check_revert(exchange,pair,inventory,log,curr_zscore,close_data)
    position = get_position(curr_zscore)
    print("\n Pair \n")
    print(pair)
    print("\n Z Score \n")

    print(curr_zscore,position)
    print("\n LOG \n")
    print(log)


    if position == "sell" and (not pair["trade_open"]):
        # place limit buy order on the second instrument  
        print("S E L L \t R A T I O")
        log,inventory = trigger_limit_buy(exchange,pair,log,inventory,close_data)
        pair["trade_open"] = True
        
    if position == "buy" and (not pair["trade_open"]):
        # place limit buy order on the second instrument 
        
        print("B U Y \t R A T I O")
        log,inventory = trigger_limit_sell(exchange,pair,log,inventory,close_data)
        pair["trade_open"] = True
       
    return stop,pair,inventory,log
    
     
    
    
 
    
    

In [21]:
def set_inventory_log(exchange):
    inventory = dict()
    log = dict()

    close_data,initial_utc_time = get_initial_data(exchange,pair)    
    inventory["balance"] = get_balance(exchange)
    inventory["current_candle_timestamp"] = initial_utc_time
    inventory["close_data"] = close_data
    inventory["num_trade"] = 0
    inventory["open_positions"] = dict() 
    inventory["filled_positions"] = dict()

  
    log["position"] = "" # long or short
    log["enter_trade"] = True
    log["exit_trade"] = False
    log["balance"] = inventory["balance"]
    log["entry_time"] = ""
    log["exit_time"] = ""
    log["connection"] = True
    log["trigger_price_S1"] = -1
    log["trigger_price_S2"] = -1
    log["trigger_amount_S1"] = -1
    log["trigger_amount_S2"] = -1
    log["filled_price_S1"] = -1
    log["filled_price_S2"] = -1
    log["trigger_time"] = ""
    log["filled_S1"] = False
    log["filled_S2"] = False

    return inventory,log


In [22]:
def set_pair_leverage(exchange,pair):
    instruments_list = [pair['name'].split("-")[0],pair['name'].split("-")[1]]
    for each in instruments_list:
        params={"leverage":pair['leverage'],"symbol":each}
        exchange.fapiPrivate_post_leverage(params)
        

In [23]:
def main(pair):
    inventory = dict()
    log = dict()
    
    exchange,connection_status = establish_connection()
    
    set_pair_leverage(exchange,pair)

    
    if connection_status== True:
        inventory,log = set_inventory_log(exchange)

        stop = False
        while(not stop):
            stop,pair,inventory,log = trade_crypto(exchange,pair,inventory,log)    
    return inventory


    
    
    

In [24]:
df = main(pair)

Connection Successful !

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': False}

 Z Score 

-1.7660616612146276 W A T C H

 LOG 

{'position': '', 'enter_trade': True, 'exit_trade': False, 'balance': 997.08872991, 'entry_time': '', 'exit_time': '', 'connection': True, 'trigger_price_S1': -1, 'trigger_price_S2': -1, 'trigger_amount_S1': -1, 'trigger_amount_S2': -1, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': '', 'filled_S1': False, 'filled_S2': False}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': False}

 Z Score 

-1.7660616612146276 W A T C H

 LOG 

{'position': '', 'enter_trade': Tru


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': False}

 Z Score 

-1.83464476916065 W A T C H

 LOG 

{'position': '', 'enter_trade': True, 'exit_trade': False, 'balance': 997.08872991, 'entry_time': '', 'exit_time': '', 'connection': True, 'trigger_price_S1': -1, 'trigger_price_S2': -1, 'trigger_amount_S1': -1, 'trigger_amount_S2': -1, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': '', 'filled_S1': False, 'filled_S2': False}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': False}

 Z Score 

-1.9740263171685157 W A T C H

 LOG 

{'position': '', 'enter_trade': True, 'exit_trade': False, 'b


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': True}

 Z Score 

-2.2386208037108357 buy

 LOG 

{'position': 'LONG', 'enter_trade': False, 'exit_trade': True, 'balance': 997.08872991, 'entry_time': datetime.datetime(2020, 3, 5, 18, 5, 27, 281257), 'exit_time': '', 'connection': True, 'trigger_price_S1': 0.2417, 'trigger_price_S2': 0.06097, 'trigger_amount_S1': 41.37360364087712, 'trigger_amount_S2': 164.0150893882237, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': datetime.datetime(2020, 3, 5, 18, 5, 25, 764773), 'filled_S1': True, 'filled_S2': True}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'tra


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': True}

 Z Score 

-2.301767497235395 buy

 LOG 

{'position': 'LONG', 'enter_trade': False, 'exit_trade': True, 'balance': 997.08872991, 'entry_time': datetime.datetime(2020, 3, 5, 18, 5, 27, 281257), 'exit_time': '', 'connection': True, 'trigger_price_S1': 0.2417, 'trigger_price_S2': 0.06097, 'trigger_amount_S1': 41.37360364087712, 'trigger_amount_S2': 164.0150893882237, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': datetime.datetime(2020, 3, 5, 18, 5, 25, 764773), 'filled_S1': True, 'filled_S2': True}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trad


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': True}

 Z Score 

-2.6022353769413744 buy

 LOG 

{'position': 'LONG', 'enter_trade': False, 'exit_trade': True, 'balance': 997.08872991, 'entry_time': datetime.datetime(2020, 3, 5, 18, 5, 27, 281257), 'exit_time': '', 'connection': True, 'trigger_price_S1': 0.2417, 'trigger_price_S2': 0.06097, 'trigger_amount_S1': 41.37360364087712, 'trigger_amount_S2': 164.0150893882237, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': datetime.datetime(2020, 3, 5, 18, 5, 25, 764773), 'filled_S1': True, 'filled_S2': True}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'tra

...


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': True}

 Z Score 

-1.5141367325339312 W A T C H

 LOG 

{'position': 'LONG', 'enter_trade': False, 'exit_trade': True, 'balance': 997.08872991, 'entry_time': datetime.datetime(2020, 3, 5, 18, 5, 27, 281257), 'exit_time': '', 'connection': True, 'trigger_price_S1': 0.2417, 'trigger_price_S2': 0.06097, 'trigger_amount_S1': 41.37360364087712, 'trigger_amount_S2': 164.0150893882237, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': datetime.datetime(2020, 3, 5, 18, 5, 25, 764773), 'filled_S1': True, 'filled_S2': True}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': True}

 Z Score 

-1.5141367325339312 W A T C H

 LOG 

{'position': 'LONG', 'enter_trade': False, 'exit_trade': True, 'balance': 997.08872991, 'entry_time': datetime.datetime(2020, 3, 5, 18, 5, 27, 281257), 'exit_time': '', 'connection': True, 'trigger_price_S1': 0.2417, 'trigger_price_S2': 0.06097, 'trigger_amount_S1': 41.37360364087712, 'trigger_amount_S2': 164.0150893882237, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': datetime.datetime(2020, 3, 5, 18, 5, 25, 764773), 'filled_S1': True, 'filled_S2': True}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': True}

 Z Score 

-1.4065284630202608 W A T C H

 LOG 

{'position': 'LONG', 'enter_trade': False, 'exit_trade': True, 'balance': 997.08872991, 'entry_time': datetime.datetime(2020, 3, 5, 18, 5, 27, 281257), 'exit_time': '', 'connection': True, 'trigger_price_S1': 0.2417, 'trigger_price_S2': 0.06097, 'trigger_amount_S1': 41.37360364087712, 'trigger_amount_S2': 164.0150893882237, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': datetime.datetime(2020, 3, 5, 18, 5, 25, 764773), 'filled_S1': True, 'filled_S2': True}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10


 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10, 'trade_open': True}

 Z Score 

-1.4065284630202608 W A T C H

 LOG 

{'position': 'LONG', 'enter_trade': False, 'exit_trade': True, 'balance': 997.08872991, 'entry_time': datetime.datetime(2020, 3, 5, 18, 5, 27, 281257), 'exit_time': '', 'connection': True, 'trigger_price_S1': 0.2417, 'trigger_price_S2': 0.06097, 'trigger_amount_S1': 41.37360364087712, 'trigger_amount_S2': 164.0150893882237, 'filled_price_S1': -1, 'filled_price_S2': -1, 'trigger_time': datetime.datetime(2020, 3, 5, 18, 5, 25, 764773), 'filled_S1': True, 'filled_S2': True}

 Pair 

{'name': 'BATUSDT-XLMUSDT', 'symbol_1': 'BAT/USDT', 'symbol_2': 'XLM/USDT', 'window': 30, 'median-win': 0.03, 'median-loss': -0.0225, 'cut-loss': 0.15, 'MA': 'simple', 'time-frame': '5m', 'leverage': 1, 'leg_size': 10

KeyboardInterrupt: 

In [ ]:
try:
    exchange_id = 'binance'
    exchange_class = getattr(ccxt, exchange_id)
    exchange = exchange_class({
        'apiKey': trading_api.binance["key"],
        'secret': trading_api.binance["secret"],
        'timeout': 30000,
        'enableRateLimit': True

    }) 
    exchange.options["defaultType"] = "future"
    print("Connection Successful !")

except():
    print("Connection NOT Successful")
    pass

In [ ]:
exchange.fapiPrivate_get_usertrades()

In [ ]:
from datetime import datetime
while(1):
    



In [ ]:
current_utc_time.second


In [ ]:
close_1 = exchange.fetch_ticker(symbol=pair['symbol_1'])["close"]
close_2 = exchange.fetch_ticker(symbol=pair['symbol_2'])["close"]
col_1 = pair['symbol_1']
col_2 = pair['symbol_2']
new_observation = pd.DataFrame({col_1:close_1,col_2:close_2})


In [ ]:
dict1= {col_1:close_1,col_2:close_2}

In [ ]:
close_data = pd.DataFrame(dict1,columns=[col_1,col_2])

In [ ]:
while(1):
    print(exchange.fetch_ticker(symbol="BNB/USDT")["close"])
    start_time = time.perf_counter()


In [ ]:
exchange

In [ ]:
exchange.fetch_ticker('BNB/USDT')

In [ ]:
exchange.fapiPrivateGetAccount()

In [ ]:
exchange.fapiPrivate_post_leverage(params)

In [ ]:
exchange.fapiPrivate_get_openorders()

In [ ]:
def create_order(self, symbol, type, side, amount, price=None,):


In [ ]:
buy_order = exchange.create_order(symbol="ADA/USDT",type="limit", side= "buy", amount=1, price=0.05213)

In [ ]:
buy_order["id"]

In [ ]:
exchange.fapiPrivate_get_order(params={'symbol': "ADAUSDT",'orderId': 50827593})

In [ ]:
exchange.fapiPrivateGetAllOrders()

In [ ]:
exchange.fetch_open_orders(symbol="ADA/USDT")

In [ ]:
cancel_order = exchange.cancel_order(symbol="ADA/USDT",id=buy_order["id"])

In [ ]:
buy_order

In [ ]:
buy_order["id"]

In [ ]:
binance_futures = ccxt.binance({ options: { defaultMarket: 'futures' } })


In [ ]:
exchange.fapiPrivateGetAllOrders(params={'symbol':'ADAUSDT'})[-1]

In [ ]:
exchange.fapiPrivate_get_allorders(params={'symbol':'ADAUSDT'})[-2]

In [ ]:
exchange.fapiPrivate_get_allorders(params={'symbol':'ADAUSDT'})[-1]

In [ ]:
exchange.fapiPrivate_get_order(params={'symbol': 'ADAUSDT','orderId': 49743919})

In [ ]:
import ccxt
from pprint import pprint
exchange = ccxt.binance({
    'enableRateLimit': True,
})
symbol = 'ETH/BTC'
ohlcv = exchange.fetch_ohlcv(symbol, '1m', 1523937720000)
print(len(ohlcv), 'candles')
print('CCXT ' + ccxt.__version__)